In [ ]:
# Instalación de paquetes
!pip install ultralytics roboflow matplotlib opencv-python-headless

import os
import matplotlib.pyplot as plt
import cv2
from ultralytics import YOLO
from roboflow import Roboflow

# Configurar Google Drive
from google.colab import drive
drive.mount('/content/drive')
drive_folder = '/content/drive/My Drive/PRUEBAS_YOLOV8'

# Crear carpeta si no existe
if not os.path.exists(drive_folder):
    os.makedirs(drive_folder)

# Descargar dataset desde Roboflow
def download_dataset(api_key="8XWwKdzZeaDFWS36mgsZ"):
    rf = Roboflow(api_key=api_key)
    project = rf.workspace("pdsei-1max5").project("classification-of-traffic-signal")
    version = project.version(8)  # Se ajusta a la versión 3 de la base de datos
    dataset = version.download("yolov8")
    return dataset.location

# Entrenamiento del modelo (ajuste a 30 épocas y corrección de argumentos)
def train_yolo_model(data_yaml, model_name="yolov8n", epochs=79, imgsz=416, batch=5, lr0=0.001):
    """
    Entrena un modelo YOLOv8 optimizado para Google Colab.
    Args:
        data_yaml: Ruta al archivo data.yaml generado por Roboflow.
        model_name: Nombre del modelo YOLO (e.g., yolov8n, yolov8s, etc.).
        epochs: Número de épocas para entrenar (ajustado para menor consumo).
        imgsz: Tamaño de las imágenes (ajustado para menor uso de memoria).
        batch: Tamaño del lote (menor para evitar sobrecarga en GPU).
        lr0: Tasa de aprendizaje inicial.
    Returns:
        Modelo entrenado y resultados.
    """
    model = YOLO(model_name)
    results = model.train(
        data=data_yaml,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        lr0=lr0,
        patience=0,  # Early stopping
        project=drive_folder,
        name="training_results_optimized",
        save=True
    )
    return model, results

# Graficar resultados
def plot_training_results(results):
    """
    Genera gráficas de métricas del entrenamiento.
    Args:
        results: Resultados del entrenamiento del modelo YOLO.
    """
    try:
        metrics = results.metrics
        epochs_range = range(len(metrics['mAP_50']))

        # Crear figura
        plt.figure(figsize=(12, 5))

        # Gráfica de mAP@50
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range, metrics['mAP_50'], label='mAP@50', color='blue')
        plt.xlabel('Épocas')
        plt.ylabel('mAP@50')
        plt.title('mAP@50 durante el Entrenamiento')
        plt.legend()

        # Gráfica de pérdidas
        plt.subplot(1, 2, 2)
        plt.plot(epochs_range, metrics['box_loss'], label='Box Loss', color='red')
        plt.plot(epochs_range, metrics['cls_loss'], label='Class Loss', color='green')
        plt.plot(epochs_range, metrics['obj_loss'], label='Object Loss', color='orange')
        plt.xlabel('Épocas')
        plt.ylabel('Pérdidas')
        plt.title('Pérdidas durante el Entrenamiento')
        plt.legend()

        # Guardar y mostrar gráficas
        plt.tight_layout()
        plt.savefig(os.path.join(drive_folder, 'training_metrics_optimized.png'))
        plt.show()
    except AttributeError:
        print("No se encontraron métricas en los resultados del entrenamiento.")

# Descargar dataset y definir ruta a data.yaml
data_yaml_path = os.path.join(download_dataset(), "data.yaml")

# Entrenar modelo con 50 épocas
model, results = train_yolo_model(data_yaml=data_yaml_path, epochs=79, imgsz=416, batch=5, lr0=0.001)

# Corregir y graficar métricas
plot_training_results(results)

# Generar video procesado (ajuste en predict)
def generate_video_with_labels(model_path, input_video_path, output_video_path):
    """
    Genera un video procesado con etiquetas detectadas por el modelo YOLO.
    Args:
        model_path: Ruta al modelo YOLO entrenado.
        input_video_path: Ruta al video de entrada.
        output_video_path: Ruta al video de salida procesado.
    """
    model = YOLO(model_path)

    # Leer video de entrada
    cap = cv2.VideoCapture(input_video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Configurar escritor de video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Realizar predicción
        results = model.predict(frame, conf=0.5) # confiabilidad 50%

        # Dibujar resultados si están disponibles
        if results and len(results[0].boxes) > 0:
            annotated_frame = results[0].plot()
        else:
            annotated_frame = frame

        # Escribir frame procesado en el video de salida
        out.write(annotated_frame)

    # Liberar recursos
    cap.release()
    out.release()

# Generar video
generate_video_with_labels(
    model_path=os.path.join(drive_folder, "training_results_optimized/weights/best.pt"),
    input_video_path="/content/drive/My Drive/PRUEBAS_YOLOV8/input_video.mp4",
    output_video_path="/content/drive/My Drive/PRUEBAS_YOLOV8/output_video.mp4"
)


Mounted at /content/drive
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Classification-of-Traffic-Signal-3 in yolov8:: 100%|██████████| 408/408 [00:00<00:00, 10262.28it/s]


100%|██████████| 6.25M/6.25M [00:00<00:00, 77.5MB/s]


Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Classification-of-Traffic-Signal-3/data.yaml, epochs=50, time=None, patience=5, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/My Drive/PRUEBAS_YOLOV8, name=training_results_optimized8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False

100%|██████████| 755k/755k [00:00<00:00, 15.3MB/s]


...

                   all         16         16      0.403      0.594      0.545      0.351
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 9, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



14 epochs completed in 0.026 hours.
Optimizer stripped from /content/drive/My Drive/PRUEBAS_YOLOV8/training_results_optimized8/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/My Drive/PRUEBAS_YOLOV8/training_results_optimized8/weights/best.pt, 6.2MB

Validating /content/drive/My Drive/PRUEBAS_YOLOV8/training_results_optimized8/weights/best.pt...
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,008,768 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  9.45it/s]


                   all         16         16      0.852      0.367      0.559      0.401
          Ceda-el-paso          1          1      0.763          1      0.995      0.506
    Curva-a-la-derecha          1          1          1          0      0.995      0.895
Interseccion-rotatoria          2          2          1          0      0.249      0.157
Proximidad-reductor-de-velocidad-tipo-resalto          1          1          1          0      0.497      0.398
           Rompemuelle          1          1          1          0          0          0
   Velocidad maxima 30          1          1      0.927          1      0.995      0.796
   Velocidad maxima 45          1          1      0.265          1      0.332      0.298
   Velocidad-maxima-55          4          4          1          0      0.682      0.414
Zona de presencia de peatones          1          1          1          0      0.142     0.0711
          Zona-escolar          3          3      0.565      0.667      0.698   

NameError: name 'plot_training_results' is not defined